In [1]:
import geopandas as gpd
import pandas as pd
import glob

In [265]:
# Read in data
profiles_df = pd.read_csv('/g/data/r78/rt1527/dea-notebooks/Waterline_extraction/validation/Avoca/photogrammetry_Xsections_Avoca.csv', skiprows=5, parse_dates=True)
profiles_df = profiles_df[profiles_df['Year/Date'] > '1987']

# Add ID column
profiles_df['site'] = profiles_df['Beach'].map(str) + '_' + profiles_df['Block'].map(str) + '_' + profiles_df['Profile'].map(str)
profiles_df.drop(['Beach', 'Block', 'Profile'], axis=1, inplace=True)

# Rename columns
profiles_df.rename({'Year/Date': 'date', 'Survey Type': 'source'}, axis=1, inplace=True)

# Compute origin and end points for each profile
origin_xy = profiles_df.loc[profiles_df.groupby(['site']).Chainage.idxmin(), ['site', 'Easting', 'Northing']]
end_xy = profiles_df.loc[profiles_df.groupby(['site']).Chainage.idxmax(), ['site', 'Easting', 'Northing']]
origin_xy.rename({'Easting': 'origin_x', 'Northing': 'origin_y'}, inplace=True, axis=1)
end_xy.rename({'Easting': 'end_x', 'Northing': 'end_y'}, inplace=True, axis=1)
end_xy.reset_index(drop=True, inplace=True)
origin_xy.reset_index(drop=True, inplace=True)

# Join origin and end points into dataframe
profiles_df = pd.merge(left=profiles_df, right=origin_xy)
profiles_df = pd.merge(left=profiles_df, right=end_xy)

# Find location and distance to water / X AHD
profiles_interp = []

for i in [0.0, 0.5, 1.0, 2.0]:

    out = profiles_df.groupby(['site', 'date']).apply(lambda x: pd.Series({f'ahd{i}_dist': interp1d(x.Elevation, x.Chainage)(i),
                                                                           f'ahd{i}_x': interp1d(x.Elevation, x.Easting)(i),
                                                                           f'ahd{i}_y': interp1d(x.Elevation, x.Northing)(i)})
                                                      
                                                      if (x.Elevation.min() < i) & (x.Elevation.max() > i) 
                                                      
                                                      else pd.Series({f'ahd{i}_dist': np.nan,
                                                                      f'ahd{i}_x': np.nan,
                                                                      f'ahd{i}_y': np.nan}))
    
    profiles_interp.append(out.astype(np.float64))

# Join values back into dataframe
shoreline_dist = profiles_df.groupby(['site', 'date']).first().join(profiles_interp)
shoreline_dist.drop(['Chainage', 'Elevation', 'Easting', 'Northing'], axis=1, inplace=True)
shoreline_dist.reset_index(inplace=True)

# Fix datetime
shoreline_dist['date'] = pd.to_datetime(shoreline_dist.date)
shoreline_dist.head()

,site,date,source,origin_x,origin_y,end_x,end_y,ahd0.0_dist,ahd0.0_x,ahd0.0_y,ahd0.5_dist,ahd0.5_x,ahd0.5_y,ahd1.0_dist,ahd1.0_x,ahd1.0_y,ahd2.0_dist,ahd2.0_x,ahd2.0_y
0,Avoca_1_1,1990-05-04,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,142.514599,354784.997669,6.295491e+06,117.200176,354769.315490,6.295471e+06,110.616143,354765.232857,6.295466e+06,91.250920,354753.234598,6.295451e+06
1,Avoca_1_1,1993-04-20,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,NaN,NaN,NaN,126.784578,354775.252871,6.295479e+06,113.447303,354766.987906,6.295468e+06,94.647256,354755.338780,6.295453e+06
2,Avoca_1_1,1996-06-30,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,129.519394,354776.946183,6.295481e+06,117.109692,354769.253958,6.295471e+06,109.194675,354764.348402,6.295465e+06,93.164786,354754.414444,6.295452e+06
3,Avoca_1_1,2001-09-13,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,NaN,NaN,NaN,121.529271,354771.996610,6.295474e+06,112.883085,354766.636441,6.295468e+06,90.007225,354752.467330,6.295450e+06
4,Avoca_1_1,2006-03-06,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,135.003382,354780.119027,6.295485e+06,128.471081,354776.092478,6.295480e+06,121.938781,354772.065929,6.295475e+06,77.750304,354744.825509,6.295440e+06


In [240]:
import re
from pyproj import Proj, transform
from shapely.geometry import Point, LineString
import geopandas as gpd
import pandas as pd
import numpy as np

# Export transects shapefile
transects_geometry = shoreline_dist.apply(lambda x: LineString([Point(x.origin_x, x.origin_y), Point(x.end_x, x.end_y)]), axis = 1)
transects_gdf = gpd.GeoDataFrame(shoreline_dist, geometry=transects_geometry)
transects_gdf.crs = 'EPSG:32756'
transects_gdf.to_file('avoca_transects2.shp')

# Export ahd0.0 water location
waterline_geometry = shoreline_dist.apply(lambda x: Point(x['ahd0.0_x'], x['ahd0.0_y']), axis = 1)
waterline_gdf = gpd.GeoDataFrame(shoreline_dist, geometry=waterline_geometry)
waterline_gdf.crs = 'EPSG:32756'
waterline_gdf.to_file('avoca_waterline.shp')




In [288]:
# Import contours
contours_gdf = gpd.read_file('output_data/avoca_utm/avoca_utm_ndwi_combined.shp')
contours_gdf['date'] = pd.to_datetime(contours_gdf.date, format='%Y')
contours_gdf.set_index('date', inplace=True)
contours_gdf

# # Extract intersections for transect 
intersect_gdf = gpd.GeoDataFrame(geometry = contours_gdf.intersection(transects_gdf.loc[5].geometry))
intersect_gdf.crs = 'EPSG:32756'
# #     intersect_gdf.reset_index().to_file('intersect_transect1.shp')

# # # Compute distances
landsat_beachwidths = intersect_gdf.distance(Point(transects_gdf.loc[5].geometry.coords[0]))
# landsat_beachwidths = landsat_beachwidths.rename('Landsat beach width (m)')
landsat_beachwidths

date
1987-01-01    154.182019
1988-01-01    149.385107
1989-01-01    154.210302
1990-01-01    146.719908
1991-01-01           NaN
1992-01-01    154.604568
1993-01-01           NaN
1994-01-01    154.070811
1995-01-01    150.428808
1996-01-01    143.396636
1997-01-01           NaN
1998-01-01    150.186459
1999-01-01           NaN
2000-01-01           NaN
2001-01-01    157.515939
2002-01-01           NaN
2003-01-01           NaN
2004-01-01    148.541558
2005-01-01    153.637097
2006-01-01    151.466395
2007-01-01    146.883186
2008-01-01    153.425930
2009-01-01    143.692049
2010-01-01    156.407317
2011-01-01    136.970818
2013-01-01    140.345696
2014-01-01    146.383001
2015-01-01    148.059847
2016-01-01    140.842944
2017-01-01    143.404038
dtype: float64

In [300]:
transects_gdf

,site,date,source,origin_x,origin_y,end_x,end_y,ahd0.0_dist,ahd0.0_x,ahd0.0_y,ahd0.5_dist,ahd0.5_x,ahd0.5_y,ahd1.0_dist,ahd1.0_x,ahd1.0_y,ahd2.0_dist,ahd2.0_x,ahd2.0_y,geometry
0,Avoca_1_1,1990-05-04,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,142.514599,354784.997669,6.295491e+06,117.200176,354769.315490,6.295471e+06,110.616143,354765.232857,6.295466e+06,91.250920,354753.234598,6.295451e+06,"LINESTRING (354696.69 6295379.04, 354795.48 62..."
1,Avoca_1_1,1993-04-20,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,NaN,NaN,NaN,126.784578,354775.252871,6.295479e+06,113.447303,354766.987906,6.295468e+06,94.647256,354755.338780,6.295453e+06,"LINESTRING (354696.69 6295379.04, 354795.48 62..."
2,Avoca_1_1,1996-06-30,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,129.519394,354776.946183,6.295481e+06,117.109692,354769.253958,6.295471e+06,109.194675,354764.348402,6.295465e+06,93.164786,354754.414444,6.295452e+06,"LINESTRING (354696.69 6295379.04, 354795.48 62..."
3,Avoca_1_1,2001-09-13,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,NaN,NaN,NaN,121.529271,354771.996610,6.295474e+06,112.883085,354766.636441,6.295468e+06,90.007225,354752.467330,6.295450e+06,"LINESTRING (354696.69 6295379.04, 354795.48 62..."
4,Avoca_1_1,2006-03-06,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,135.003382,354780.119027,6.295485e+06,128.471081,354776.092478,6.295480e+06,121.938781,354772.065929,6.295475e+06,77.750304,354744.825509,6.295440e+06,"LINESTRING (354696.69 6295379.04, 354795.48 62..."
5,Avoca_1_1,2008-07-03,Stereo photogrammtery,354696.69,6295379.04,354795.48,6295504.62,133.940787,354779.468845,6.295484e+06,121.137811,354771.570036,6.295474e+06,115.819071,354768.295506,6.295470e+06,91.489061,354753.299828,6.295451e+06,"LINESTRING (354696.69 6295379.04, 354795.48 62..."
6,Avoca_1_1,2016-06-07,LiDar,354696.69,6295379.04,354795.48,6295504.62,NaN,NaN,NaN,NaN,NaN,NaN,110.628900,354765.104000,6.295466e+06,91.736202,354753.457798,6.295451e+06,"LINESTRING (354696.69 6295379.04, 354795.48 62..."
7,Avoca_1_2,1990-05-04,Stereo photogrammtery,354681.00,6295391.43,354761.18,6295493.25,NaN,NaN,NaN,111.086099,354749.831030,6.295479e+06,105.493153,354746.365825,6.295474e+06,83.105312,354732.491729,6.295457e+06,"LINESTRING (354681 6295391.43, 354761.18 62954..."
8,Avoca_1_2,1993-04-20,Stereo photogrammtery,354681.00,6295391.43,354761.18,6295493.25,NaN,NaN,NaN,111.122431,354749.854293,6.295479e+06,104.505557,354745.753557,6.295473e+06,88.832393,354736.040457,6.295461e+06,"LINESTRING (354681 6295391.43, 354761.18 62954..."
9,Avoca_1_2,1996-06-30,Stereo photogrammtery,354681.00,6295391.43,354761.18,6295493.25,114.065707,354751.681767,6.295481e+06,108.083057,354747.975142,6.295476e+06,102.100407,354744.268517,6.295472e+06,85.652277,354734.077675,6.295459e+06,"LINESTRING (354681 6295391.43, 354761.18 62954..."
